In [2]:
#%%
import pandas as pd
import numpy as np
import xlwings as xw
import os
from pathlib import Path
import secret

In [2]:
df = pd.DataFrame(data=np.random.rand(100, 10),
                  columns=[f"Trial {i}" for i in range(1,11)])
df

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Trial 6,Trial 7,Trial 8,Trial 9,Trial 10
0,0.537714,0.395904,0.254964,0.766267,0.229634,0.394690,0.009831,0.187724,0.095451,0.799392
1,0.633684,0.888213,0.852097,0.846546,0.384072,0.521728,0.207009,0.700369,0.589809,0.148578
2,0.267482,0.790306,0.923548,0.050137,0.071255,0.591444,0.838329,0.871431,0.244580,0.725733
3,0.655501,0.557914,0.147443,0.050624,0.704691,0.497086,0.446583,0.236061,0.914638,0.881558
4,0.307731,0.458860,0.629872,0.351644,0.204521,0.899731,0.001713,0.194388,0.193709,0.132937
...,...,...,...,...,...,...,...,...,...,...
95,0.280088,0.841595,0.541563,0.435985,0.771251,0.557137,0.059771,0.912933,0.054726,0.263561
96,0.419841,0.193772,0.459731,0.612408,0.785720,0.952739,0.768493,0.496531,0.287387,0.155079
97,0.888015,0.062935,0.967158,0.747860,0.540372,0.081342,0.907416,0.698333,0.180955,0.884668
98,0.035255,0.806135,0.089489,0.114813,0.751401,0.154042,0.895026,0.473134,0.334769,0.899637


In [4]:
xw.view(df)


In [18]:
downloads = '/Users/infowizard/Downloads/'
file = 'J12539 HBA - OKWOK FPSO - PABX PAGA R0.xlsm'
file_path = Path(downloads, file)
wb = xw.Book(file_path, password='BP85')

In [19]:
visible_sheets = [sht.name for sht in wb.sheets if sht.visible]

In [20]:
[item for item in visible_sheets]

['FX', 'Cover', 'Intro', 'PABX', 'PAGA', 'ES', 'T&C']

In [21]:
# Read and set currency from FX sheet
fx = wb.sheets['FX']
exchange_rates = dict(fx.range('A2:B9').value)
exchange_rates

{'AUD': 0.96,
 'EUR': 1.47,
 'GBP': 1.67,
 'JPY': 0.0104,
 'NOK': 0.1363,
 'DKK': 0.1977,
 'SGD': 1.0,
 'USD': 1.35}

In [22]:
# Currency to quote
quoted_currency = fx.range('B12').value
quoted_currency

'USD'

In [23]:
# Get project info
project_info = dict(fx.range('A36:B45').value)
project_info

{'Attend to: ': 'Rahul Prasad',
 'Designation: ': None,
 'Client Name: ': 'HBA Offshore Pte Ltd',
 'Client RFQ No: ': 'As per email attachments',
 'Ref Doc No: ': None,
 'Project Name: ': 'OKWOK FPSO - PABX PAGA',
 'Prepared By: ': 'Sahib',
 'Sales Manager: ': 'Shaun Teo',
 'Jason Ref: ': 'J12539',
 'Revision Num: ': 'R0'}

In [7]:
paga = wb.sheets['PAGA']
paga.range('D2').value = project_info['Jason Ref: ']

In [62]:
cols = ['No', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal', 'Part', 'Cur', 'Cost', 'SubCost', 'Discount', 'Category','System']
systems = pd.DataFrame(cols)
defaults = {}
skip_sheets = ['FX', 'Cover', 'Intro', 'ES', 'T&C']
for sheet in visible_sheets:
    if sheet not in skip_sheets:
        ws = wb.sheets[sheet]
        escalation = dict(ws.range('K2:L5').value)
        default_mu = ws.range('H5')
        escalation['default_mu'] = default_mu
        defaults[sheet] = escalation
        last_row = ws.range('D100000').end('up').row  #Returns a number
        last_cell = 'K' + str(last_row) 
        data = ws.range('A8:' + last_cell).options(pd.DataFrame, index=False).value
        data.columns = cols
        systems = pd.concat([systems, data], join='outer')
systems

ValueError: Length mismatch: Expected axis has 11 elements, new values have 13 elements

In [54]:
ws.range('A8:' + last_cell).options(pd.DataFrame, index=False, columns=cols).value

,None,None,None,None,None,None,None,None,None,None,None
0,1.0,NaN,None,PABX,NaN,NaN,None,None,None,None,NaN
1,None,NaN,None,None,NaN,NaN,Start Line: Delete Forbidden,None,None,None,NaN
2,1.01,1.0,Lot,PABX System Equipment,NaN,NaN,None,None,True,True,NaN
3,None,NaN,None,None,NaN,NaN,None,None,None,None,NaN
4,None,NaN,None,Main PABX Cabinet & Controller,NaN,NaN,None,None,None,None,NaN
5,None,NaN,None,None,NaN,NaN,None,None,None,None,NaN
6,1.01.01,1.0,set,"MRC MCX 5000(D), AC 220V & DC24V PSTN-96 Lines...",9239.0,9239.0,MCX5000D/96,USD,6900.0,6900.0,0.1
7,None,NaN,None,None,NaN,NaN,None,None,None,None,NaN
8,None,NaN,None,*** MRC MCX5000D will replace an existing PABX...,NaN,NaN,None,None,None,None,NaN
9,None,NaN,None,***Exisitng telephones can be re-used to conne...,NaN,NaN,None,None,None,None,NaN


In [32]:
escalations = {}
escalation = dict(paga.range('K2:L5').value)
default = 0.5
escalation['MU'] = default
escalations['PAGA'] = escalation
escalations['PAGA']['Default']

0.03